In [1]:
#download DOW30 data from yahoo finance
import yfinance as yf
import pandas as pd

#dow30_tickers = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']
dow30_tickers = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'CRM', 'TRV', 'UNH', 'VZ', 'V', 'WBA', 'WMT', 'DIS']

dow30 = yf.download(dow30_tickers, start="2010-01-01", end="2021-12-31")

[*********************100%***********************]  29 of 29 completed


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
dow30 = dow30['Adj Close']

In [4]:
dow30.index = pd.to_datetime(dow30.index)

In [5]:
import numpy as np

In [6]:
portfolio_money = 10000

In [7]:
def calculate_max_drawdown(portfolio_values):
    max_drawdown = 0
    peak_value = portfolio_values[0]

    for value in portfolio_values:
        if value > peak_value:
            peak_value = value
        else:
            drawdown = (peak_value - value) / peak_value
            max_drawdown = max(max_drawdown, drawdown)

    return max_drawdown

In [8]:
def calculate_sharpe_ratio(portfolio_values, risk_free_rate = 0.0):
    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    average_return = np.mean(returns)
    std_deviation = np.std(returns)
    sharpe_ratio = (average_return - risk_free_rate) / std_deviation
    return sharpe_ratio

In [27]:
def calculate_portfolio_values_over_time(data, columns, money= portfolio_money):
    partial_money = money / len(columns)
    portfolio_values = np.zeros(len(data))
    temp = np.array([])
    for column in columns:
        num_of_shares = partial_money // data[column][0]
        for index, row in data.iterrows():
            temp = np.append(temp, num_of_shares * row[column])
        
        portfolio_values = portfolio_values + temp
        temp = np.array([])
    return portfolio_values

In [22]:
# Strategy-1 

"""
Dow 30’un tamamını eşit ağırlıklı olacak şekilde al, yani 30 hisse varsa herbirinden 1/30 kadar alın.
"""

data = dow30.copy()

dataframe_1 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    start_value = 0
    end_value = 0
    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'
    temp = data.loc[start_date:end_date]

    portfolio_values = calculate_portfolio_values_over_time(temp, temp.columns, portfolio_money)
    start_value = portfolio_values[0]
    end_value = portfolio_values[-1]

    annual_return = (end_value - start_value) / start_value
    dataframe_1['Annual Return'][str(i)] = annual_return

    dataframe_1['Std Dev'][str(i)] = np.std(portfolio_values)
    dataframe_1['Max Drawdown'][str(i)] = calculate_max_drawdown(portfolio_values)
    dataframe_1['Sharpe Ratio'][str(i)] = calculate_sharpe_ratio(portfolio_values)

dataframe_1['Annual Return']['Average'] = dataframe_1['Annual Return'].mean()
dataframe_1['Std Dev']['Average'] = dataframe_1['Std Dev'].mean()
dataframe_1['Max Drawdown']['Average'] = dataframe_1['Max Drawdown'].mean()
dataframe_1['Sharpe Ratio']['Average'] = dataframe_1['Sharpe Ratio'].mean()

dataframe_1

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio
2015,0.044783,250.786419,0.122017,0.022511
2016,0.149551,515.238303,0.079457,0.074439
2017,0.284422,656.639972,0.027192,0.25295
2018,0.026256,354.70092,0.154936,0.014956
2019,0.269353,519.450932,0.062011,0.129213
2020,0.089874,673.301865,0.324176,0.026724
2021,0.218656,410.330082,0.06321,0.107567
Average,0.154699,482.921213,0.119,0.089766


In [26]:
#2 

"""
Bir önceki sene en iyi getiri sağlayan 1 hisseyi alın. Sene boyunca sadece bu hisseyi tutun.
"""

data = dow30.copy()

dataframe_2 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Best Stock'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    best_stock = None
    best_return = 0

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        if temp_return > best_return:
            best_return = temp_return
            best_stock = stock
    
    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date]

    stocks = [best_stock]
    portfolio_values = calculate_portfolio_values_over_time(temp_data, stocks, portfolio_money)

    dataframe_2.loc[str(i), 'Annual Return'] = (portfolio_values[-1] - portfolio_values[0]) / portfolio_values[0]
    dataframe_2.loc[str(i), 'Std Dev'] = np.std(portfolio_values)
    dataframe_2.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_2.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)
    dataframe_2.loc[str(i), 'Best Stock'] = best_stock

    

dataframe_2.loc['Average', 'Annual Return'] = dataframe_2['Annual Return'].mean()
dataframe_2.loc['Average', 'Std Dev'] = dataframe_2['Std Dev'].mean()
dataframe_2.loc['Average', 'Max Drawdown'] = dataframe_2['Max Drawdown'].mean()
dataframe_2.loc['Average', 'Sharpe Ratio'] = dataframe_2['Sharpe Ratio'].mean()

dataframe_2

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Best Stock
2015,-0.024022,656.321935,0.283067,0.001378,INTC
2016,-0.163896,580.610801,0.231071,-0.043659,NKE
2017,0.727058,2006.633877,0.077055,0.177086,CAT
2018,0.107767,600.075174,0.246703,0.030613,BA
2019,0.235978,618.55835,0.121369,0.078839,MRK
2020,0.782399,2935.965129,0.314273,0.092901,AAPL
2021,0.385508,1133.27017,0.185989,0.090713,AAPL
Average,0.29297,1218.776491,0.208504,0.061124,NaN


In [30]:
# 3

"""
Bir önceki sene en kötü getiri sağlayan 1 hisseyi alın. Sene boyunca sadece bu hisseyi tutun.
"""

data = dow30.copy()

dataframe_3 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Worst Stock'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    worst_stock = None
    worst_return =  10000000

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        if temp_return < worst_return:
            worst_return = temp_return
            worst_stock = stock
    
    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date]
    stocks = [worst_stock]
    portfolio_values = calculate_portfolio_values_over_time(temp_data, stocks)
    dataframe_3.loc[str(i), 'Annual Return'] = (portfolio_values[-1] - portfolio_values[0]) / portfolio_values[0]
    dataframe_3.loc[str(i), 'Std Dev'] = np.std(portfolio_values)
    dataframe_3.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_3.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)
    dataframe_3.loc[str(i), 'Worst Stock'] = worst_stock

    
dataframe_3.loc['Average', 'Annual Return'] = dataframe_3['Annual Return'].mean()
dataframe_3.loc['Average', 'Std Dev'] = dataframe_3['Std Dev'].mean()
dataframe_3.loc['Average', 'Max Drawdown'] = dataframe_3['Max Drawdown'].mean()
dataframe_3.loc['Average', 'Sharpe Ratio'] = dataframe_3['Sharpe Ratio'].mean()

dataframe_3

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Worst Stock
2015,-0.123078,657.293601,0.22959,-0.032576,IBM
2016,0.157291,554.054911,0.093001,0.053972,WMT
2017,0.21941,631.161114,0.151901,0.064677,NKE
2018,-0.066457,849.746225,0.247298,-0.007162,WBA
2019,0.364123,774.266431,0.119654,0.091038,GS
2020,-0.295169,884.331136,0.419543,-0.030237,WBA
2021,-0.000049,833.530077,0.300903,0.011111,BA
Average,0.036582,740.626214,0.223127,0.021546,NaN


In [32]:
#4

"""
Bir önceki sene en iyi getiri sağlayan 3 hisseyi alın. Sene boyunca bu 3 hisseyi tutun.
"""

data = dow30.copy()

dataframe_4 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Best Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    best_stock_1 = None
    best_return_1 = 0
    best_stock_2 = None
    best_return_2 = 0
    best_stock_3 = None
    best_return_3 = 0

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        if temp_return > best_return_1:
            best_return_3 = best_return_2
            best_stock_3 = best_stock_2
            best_return_2 = best_return_1
            best_stock_2 = best_stock_1
            best_return_1 = temp_return
            best_stock_1 = stock
        elif temp_return > best_return_2:
            best_return_3 = best_return_2
            best_stock_3 = best_stock_2
            best_return_2 = temp_return
            best_stock_2 = stock
        elif temp_return > best_return_3:
            best_return_3 = temp_return
            best_stock_3 = stock
        
    best_stocks = [best_stock_1, best_stock_2, best_stock_3]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date, best_stocks]

    portfolio = calculate_portfolio_values_over_time(temp_data, best_stocks)

    dataframe_4.loc[str(i), 'Annual Return'] = (portfolio[-1] - portfolio[0]) / portfolio[0]
    dataframe_4.loc[str(i), 'Std Dev'] = np.std(portfolio)
    dataframe_4.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio)
    dataframe_4.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio)
    dataframe_4.loc[str(i), 'Best Stocks'] = best_stocks

    

dataframe_4.loc['Average', 'Annual Return'] = dataframe_4['Annual Return'].mean()
dataframe_4.loc['Average', 'Std Dev'] = dataframe_4['Std Dev'].mean()
dataframe_4.loc['Average', 'Max Drawdown'] = dataframe_4['Max Drawdown'].mean()
dataframe_4.loc['Average', 'Sharpe Ratio'] = dataframe_4['Sharpe Ratio'].mean()

dataframe_4

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Best Stocks
2015,-0.003233,366.442466,0.181763,0.005643,"[INTC, AAPL, AMGN]"
2016,-0.067719,360.151235,0.146799,-0.019395,"[NKE, CRM, MCD]"
2017,0.454603,1300.257379,0.055113,0.201523,"[CAT, UNH, JPM]"
2018,-0.047907,595.96065,0.293655,-0.003731,"[BA, CAT, AAPL]"
2019,0.339324,733.835414,0.064334,0.113475,"[MRK, CRM, MSFT]"
2020,0.372783,1518.602249,0.330908,0.060577,"[AAPL, MSFT, JPM]"
2021,0.385592,1226.512906,0.093374,0.115175,"[AAPL, MSFT, NKE]"
Average,0.204777,871.680329,0.166564,0.06761,NaN


In [34]:
#5

"""
Bir önceki sene en kötü getiri sağlayan 3 hisseyi alın. Sene boyunca bu 3 hisseyi tutun.
"""


data = dow30.copy()

dataframe_5 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'worst Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    worst_stock_1 = None
    worst_return_1 = 100000000
    worst_stock_2 = None
    worst_return_2 = 100000000
    worst_stock_3 = None
    worst_return_3 = 100000000

    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]

        if temp_return < worst_return_1:
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = worst_return_1
            worst_stock_2 = worst_stock_1
            worst_return_1 = temp_return
            worst_stock_1 = stock   
        elif temp_return < worst_return_2:
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = temp_return
            worst_stock_2 = stock  
        elif temp_return < worst_return_3:
            worst_return_3 = temp_return
            worst_stock_3 = stock
            


    worst_stocks = [worst_stock_1, worst_stock_2, worst_stock_3]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date]

    portfolio_values = calculate_portfolio_values_over_time(temp_data, worst_stocks)

    dataframe_5.loc[str(i), 'Annual Return'] = (portfolio[-1] - portfolio[0]) / portfolio[0]
    dataframe_5.loc[str(i), 'Std Dev'] = np.std(portfolio_values)
    dataframe_5.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_5.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)
    dataframe_5.loc[str(i), 'worst Stocks'] = worst_stocks
    

dataframe_5.loc['Average', 'Annual Return'] = dataframe_5['Annual Return'].mean()
dataframe_5.loc['Average', 'Std Dev'] = dataframe_5['Std Dev'].mean()
dataframe_5.loc['Average', 'Max Drawdown'] = dataframe_5['Max Drawdown'].mean()
dataframe_5.loc['Average', 'Sharpe Ratio'] = dataframe_5['Sharpe Ratio'].mean()

dataframe_5

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,worst Stocks
2015,0.385592,541.529412,0.218391,-0.011126,"[IBM, CVX, BA]"
2016,0.385592,886.095592,0.095707,0.086811,"[WMT, AXP, CAT]"
2017,0.385592,680.706389,0.064627,0.140285,"[NKE, CRM, AMGN]"
2018,0.385592,555.823309,0.155033,0.0183,"[WBA, IBM, MRK]"
2019,0.385592,557.940779,0.134174,0.082994,"[GS, IBM, CAT]"
2020,0.385592,978.626685,0.426568,-0.020784,"[WBA, MMM, BA]"
2021,0.385592,609.029826,0.167863,0.073245,"[BA, WBA, CVX]"
Average,0.385592,687.107428,0.180337,0.052818,NaN


In [35]:
#6

"""
Bir önceki sene en iyi getiri sağlayan 5 hisseyi alın. Sene boyunca bu 5 hisseyi tutun.
"""


data = dow30.copy()

dataframe_6 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'Best Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    best_stock_1 = None
    best_return_1 = 0
    best_stock_2 = None
    best_return_2 = 0
    best_stock_3 = None
    best_return_3 = 0
    best_return_4 = 0
    best_stock_4 = None
    best_return_5 = 0
    best_stock_5 = None


    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]
        
        if temp_return > best_return_1:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = best_stock_3
            best_return_4 = best_return_3
            best_stock_3 = best_stock_2
            best_return_3 = best_return_2
            best_stock_2 = best_stock_1
            best_return_2 = best_return_1
            best_stock_1 = stock
            best_return_1 = temp_return
        elif temp_return > best_return_2:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = best_stock_3
            best_return_4 = best_return_3
            best_stock_3 = best_stock_2
            best_return_3 = best_return_2
            best_stock_2 = stock
            best_return_2 = temp_return
        elif temp_return > best_return_3:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = best_stock_3
            best_return_4 = best_return_3
            best_stock_3 = stock
            best_return_3 = temp_return
        elif temp_return > best_return_4:
            best_stock_5 = best_stock_4
            best_return_5 = best_return_4
            best_stock_4 = stock
            best_return_4 = temp_return
        elif temp_return > best_return_5:
            best_stock_5 = stock
            best_return_5 = temp_return
            
        
    best_stocks = [best_stock_1, best_stock_2, best_stock_3, best_stock_4, best_stock_5]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date]

    portfolio_values = calculate_portfolio_values_over_time(temp_data, best_stocks)

    dataframe_6.loc[str(i), 'Annual Return'] = (portfolio[-1] - portfolio[0]) / portfolio[0]
    dataframe_6.loc[str(i), 'Std Dev'] = np.std(portfolio_values)
    dataframe_6.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_6.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)
    dataframe_6.loc[str(i), 'Best Stocks'] = best_stocks

    

dataframe_6.loc['Average', 'Annual Return'] = dataframe_6['Annual Return'].mean()
dataframe_6.loc['Average', 'Std Dev'] = dataframe_6['Std Dev'].mean()
dataframe_6.loc['Average', 'Max Drawdown'] = dataframe_6['Max Drawdown'].mean()
dataframe_6.loc['Average', 'Sharpe Ratio'] = dataframe_6['Sharpe Ratio'].mean()

dataframe_6


,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,Best Stocks
2015,0.385592,315.35362,0.141184,0.025884,"[INTC, AAPL, AMGN, UNH, WBA]"
2016,0.385592,291.452398,0.137846,0.00527,"[NKE, CRM, MCD, HD, MSFT]"
2017,0.385592,917.793957,0.07412,0.15391,"[CAT, UNH, JPM, CVX, GS]"
2018,0.385592,415.710604,0.199116,-0.000688,"[BA, CAT, AAPL, MCD, WMT]"
2019,0.385592,775.007956,0.067559,0.131505,"[MRK, CRM, MSFT, NKE, V]"
2020,0.385592,1195.338891,0.316445,0.05431,"[AAPL, MSFT, JPM, V, PG]"
2021,0.385592,877.100818,0.071883,0.106642,"[AAPL, MSFT, NKE, CRM, CAT]"
Average,0.385592,683.965464,0.144022,0.068119,NaN


In [36]:
#7

"""
Bir önceki sene en kötü getiri sağlayan 5 hisseyi alın. Sene boyunca bu 5 hisseyi tutun.
"""


data = dow30.copy()

dataframe_7 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio', 'worst Stocks'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    worst_stock_1 = None
    worst_return_1 = 100000000
    worst_stock_2 = None
    worst_return_2 = 100000000
    worst_stock_3 = None
    worst_return_3 = 100000000
    worst_return_4 = 100000000
    worst_stock_4 = None
    worst_return_5 = 100000000
    worst_stock_5 = None


    for stock in data.columns:
        start_date = str(i-1) + '-01-01'
        end_date = str(i-1) + '-12-31'
        temp_data = data.loc[start_date:end_date, stock]
        temp_return = (temp_data[-1] - temp_data[0]) / temp_data[0]

        if temp_return < worst_return_1:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = worst_return_3
            worst_stock_4 = worst_stock_3
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = worst_return_1
            worst_stock_2 = worst_stock_1
            worst_return_1 = temp_return
            worst_stock_1 = stock
        elif temp_return < worst_return_2:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = worst_return_3
            worst_stock_4 = worst_stock_3
            worst_return_3 = worst_return_2
            worst_stock_3 = worst_stock_2
            worst_return_2 = temp_return
            worst_stock_2 = stock
        elif temp_return < worst_return_3:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = worst_return_3
            worst_stock_4 = worst_stock_3
            worst_return_3 = temp_return
            worst_stock_3 = stock
        elif temp_return < worst_return_4:
            worst_return_5 = worst_return_4
            worst_stock_5 = worst_stock_4
            worst_return_4 = temp_return
            worst_stock_4 = stock
        elif temp_return < worst_return_5:
            worst_return_5 = temp_return
            worst_stock_5 = stock
            


    worst_stocks = [worst_stock_1, worst_stock_2, worst_stock_3, worst_stock_4, worst_stock_5]

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'

    while start_date not in data.index:
        start_date = pd.to_datetime(start_date) + pd.DateOffset(days=1)
        start_date = start_date.strftime('%Y-%m-%d')
    
    while end_date not in data.index:
        end_date = pd.to_datetime(end_date) + pd.DateOffset(days=-1)
        end_date = end_date.strftime('%Y-%m-%d')

    temp_data = data.loc[start_date:end_date, worst_stocks]
    
    portfolio_values = calculate_portfolio_values_over_time(temp_data, worst_stocks)

    dataframe_7.loc[str(i), 'Annual Return'] = (portfolio[-1] - portfolio[0]) / portfolio[0]
    dataframe_7.loc[str(i), 'Std Dev'] = np.std(portfolio_values)
    dataframe_7.loc[str(i), 'Max Drawdown'] = calculate_max_drawdown(portfolio_values)
    dataframe_7.loc[str(i), 'Sharpe Ratio'] = calculate_sharpe_ratio(portfolio_values)
    dataframe_7.loc[str(i), 'worst Stocks'] = worst_stocks

dataframe_7.loc['Average', 'Annual Return'] = dataframe_7['Annual Return'].mean()
dataframe_7.loc['Average', 'Std Dev'] = dataframe_7['Std Dev'].mean()
dataframe_7.loc['Average', 'Max Drawdown'] = dataframe_7['Max Drawdown'].mean()
dataframe_7.loc['Average', 'Sharpe Ratio'] = dataframe_7['Sharpe Ratio'].mean()

dataframe_7

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio,worst Stocks
2015,0.385592,360.484984,0.16726,0.020388,"[IBM, CVX, BA, VZ, MCD]"
2016,0.385592,930.644478,0.097231,0.104216,"[WMT, AXP, CAT, CVX, IBM]"
2017,0.385592,419.661649,0.049549,0.117839,"[NKE, CRM, AMGN, KO, WBA]"
2018,0.385592,575.480653,0.139079,0.021776,"[WBA, IBM, MRK, VZ, DIS]"
2019,0.385592,407.211952,0.135115,0.063838,"[GS, IBM, CAT, MMM, CVX]"
2020,0.385592,791.703819,0.399247,-0.021459,"[WBA, MMM, BA, CVX, VZ]"
2021,0.385592,423.457775,0.127544,0.064039,"[BA, WBA, CVX, INTC, MRK]"
Average,0.385592,558.377902,0.159289,0.052948,NaN


In [69]:
#8
"""
Bir önceki yılın korelasyon matrisi değerlerine göre aralarında korelasyonları en düşük olan
(en çok negatif olanlar arasında) 2 hisseden oluşan bir portföy oluşturun. Bunların
portföydeki ağırlıklarını ayrı ayrı deneyerek ders slaytlarında gördüğümüz şekilde bir getiri ve
standart sapma tablosu oluşturun, ayrıca portfolio frontier’in gösterildiği grafiği de çizin. 
"""

data = dow30.copy()

dataframe_8 = pd.DataFrame(columns = ['Annual Return', 'Std Dev', 'Max Drawdown', 'Sharpe Ratio'], 
index = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', 'Average'])

for i in range(2015, 2022):
    start_date = str(i-1) + '-01-01'
    end_date = str(i-1) + '-12-31'
    temp_data = data.loc[start_date:end_date]

    # calculate correlation between stocks
    temp_data = pd.DataFrame(temp_data)
    corr_matrix = temp_data.corr()

    # find the most negatively correlated columns
    min_corr_col_1 = None
    min_corr_col_2 = None
    min_corr = 1

    for col in corr_matrix.columns:
        for row in corr_matrix.index:
            if col != row and corr_matrix[col][row] < min_corr:
                min_corr = corr_matrix[col][row]
                min_corr_col_1 = col
                min_corr_col_2 = row

    # create a portfolio with the most negatively correlated stocks

    start_date = str(i) + '-01-01'
    end_date = str(i) + '-12-31'
    temp_data = data.loc[start_date:end_date]

    max_return = -10000000
    best_w1 = 0
    best_w2 = 0

    

    for a in np.arange(0,1,0.05):
        w1 = a
        w2 = 1 - a
        temp_data['Portfolio'] = temp_data[min_corr_col_1] * w1 + temp_data[min_corr_col_2] * w2
        
        if temp_data['Portfolio'][-1] - temp_data['Portfolio'][0] > max_return:
            max_return = temp_data['Portfolio'][-1] - temp_data['Portfolio'][0]
            best_w1 = w1
            best_w2 = w2 
        

    temp_data['Portfolio'] = temp_data[min_corr_col_1] * best_w1 + temp_data[min_corr_col_2] * best_w2

    dataframe_8['Annual Return'][str(i)] = (temp_data['Portfolio'][-1] - temp_data['Portfolio'][0]) / temp_data['Portfolio'][0]
    dataframe_8['Std Dev'][str(i)] = temp_data['Portfolio'].std()
    dataframe_8['Max Drawdown'][str(i)] = calculate_max_drawdown(temp_data['Portfolio'].values)
    dataframe_8['Sharpe Ratio'][str(i)] = calculate_sharpe_ratio(temp_data['Portfolio'].values)

dataframe_8['Annual Return']['Average'] = dataframe_8['Annual Return'].mean()
dataframe_8['Std Dev']['Average'] = dataframe_8['Std Dev'].mean()
dataframe_8['Max Drawdown']['Average'] = dataframe_8['Max Drawdown'].mean()
dataframe_8['Sharpe Ratio']['Average'] = dataframe_8['Sharpe Ratio'].mean()

dataframe_8

,Annual Return,Std Dev,Max Drawdown,Sharpe Ratio
2015,0.178697,4.718589,0.121763,0.054072
2016,0.157292,2.937341,0.093002,0.053972
2017,0.252748,5.554809,0.116068,0.095822
2018,-0.024049,4.906666,0.207931,0.00101
2019,0.361454,11.650739,0.117134,0.091862
2020,0.041712,12.806233,0.410665,0.020965
2021,0.251871,2.425794,0.184229,0.065845
Average,0.174246,6.428596,0.178684,0.054793


8. SORUDA EKSİK GRAFİK VAR.